In [ ]:
%load_ext autoreload
%autoreload 2
import waffles
import numpy as np
import json
import shutil 
from tqdm import tqdm
import mplhep
import matplotlib.pyplot as plt
mplhep.style.use(mplhep.style.ROOT)
plt.rcParams.update({'font.size': 20,
                        'grid.linestyle': '--',
                        'axes.grid': True,
                        'figure.autolayout': True,
                        'figure.figsize': [14,6]
                        })

import waffles
import numpy as np
import json
import shutil 
from tqdm import tqdm
from pathlib import Path

from waffles.input_output.hdf5_structured import load_structured_waveformset
from waffles.data_classes.Waveform import Waveform
from waffles.data_classes.WaveformSet import WaveformSet
from waffles.data_classes.BasicWfAna import BasicWfAna
from waffles.data_classes.IPDict import IPDict
from waffles.data_classes.UniqueChannel import UniqueChannel
from waffles.data_classes.ChannelWsGrid import ChannelWsGrid
from waffles.utils.baseline.baseline import SBaseline
from waffles.np02_utils.AutoMap import generate_ChannelMap, dict_uniqch_to_module, dict_module_to_uniqch, strUch, ordered_channels_membrane, ordered_channels_cathode
from waffles.np02_utils.PlotUtils import np02_gen_grids, plot_grid, plot_detectors, genhist, fithist, runBasicWfAnaNP02
from waffles.np02_utils.PlotUtils import matplotlib_plot_WaveformSetGrid
from waffles.np02_utils.load_utils import ch_read_calib, open_processed
from utils import apply_derivativeWf, apply_moving_averageWf, apply_full_peak_search, apply_peak_search_from_derivative_2, find_peaks_from_derivative_2
from utils import compute_all_peaks, plot_counts, plot_runs, plot_ratio, load_data_search_peaks

In [ ]:
dettype = "cathode"
rates_results = {}


## Only change if necessary
datadir = f"/eos/experiment/neutplatform/protodune/experiments/ProtoDUNE-VD/commissioning/"
det = "VD_Cathode_PDS" if dettype == "cathode" else "VD_Membrane_PDS"
endpoint = 106 if dettype == "cathode" else 107
listofch = ordered_channels_cathode if endpoint==106 else ordered_channels_membrane
validchannels = {}
validchannels[106] = ordered_channels_cathode 
validchannels[107] = ordered_channels_membrane
subwaves = 1000 if dettype == "cathode" else 20000

# Way to low... keep scrollng
dletter = dettype.upper()[0] # C or M...
group1 = [ f"{dletter}{detnum}({chnum})" for detnum in range(1, 3) for chnum in range(1,3) ]
group2 = [ f"{dletter}{detnum}({chnum})" for detnum in range(3, 5) for chnum in range(1,3) ]
group3 = [ f"{dletter}{detnum}({chnum})" for detnum in range(5, 7) for chnum in range(1,3) ]
group4 = [ f"{dletter}{detnum}({chnum})" for detnum in range(7, 9) for chnum in range(1,3) ]
groupall = group1+group2+group3+group4

calib_values = ch_read_calib()
list_of_unch = ordered_channels_cathode

In [ ]:
# run = 42368
run = 42410
# run = 38670
# run = 37249

wfsetsub = load_data_search_peaks(run, dettype, datadir, endpoint, nwaveforms=None, rates_results=rates_results, subwaves=subwaves)



In [ ]:
wfsetcath = open_processed(run, "cathode", datadir, channels=None, endpoints = [106], nwaveforms=10, verbose=True, mergefiles=True, file_slice=slice(0,1))

In [ ]:
def remove_weird_channels(waveform: Waveform, validchannels = []) -> bool:
    if waveform.channel not in validchannels:
        return False
    
    return True

print(wfsetsub)
wfsetsub= WaveformSet.from_filtered_WaveformSet(wfsetsub, remove_weird_channels, validchannels[endpoint], show_progress=True)
print(wfsetsub)

In [ ]:
runBasicWfAnaNP02(wfsetsub, onlyoptimal=False, baselinestart=0, baselinefinish=64, int_ll=64, int_ul=200, amp_ll=64, amp_ul=200, threshold=25, configyaml="")

In [ ]:
if dettype == "membrane":
    time_bins=wfsetsub.points_per_wf//2
else:
    time_bins=1024
    
argsheat = dict(
    mode="heatmap",
    analysis_label="std",
    adc_range_above_baseline=500,
    adc_range_below_baseline=-200,
    adc_bins=200,
    time_bins=time_bins,
    time_range_lower_limit=0,
    time_range_upper_limit=1024,
    filtering=0,
    share_y_scale=False,
    share_x_scale=True,
    wfs_per_axes=1000,
    zlog=True
)
detector=group1
plot_detectors(wfsetsub, detector, **argsheat)

detector=group2
plot_detectors(wfsetsub, detector, **argsheat)

detector=group3
plot_detectors(wfsetsub, detector, **argsheat)

detector=group4
plot_detectors(wfsetsub, detector, **argsheat)

In [ ]:
wfsetch = ChannelWsGrid.clusterize_waveform_set(wfsetsub)

In [ ]:
# timestamps = sorted([ (w.timestamp - w.daq_window_timestamp, w.channel, w.record_number) for i, w in enumerate(wfsetch[endpoint][17].waveforms) if w], key= lambda x: x[0])
timestamps = sorted([ (w.timestamp - w.daq_window_timestamp, w.channel, w.record_number) for i, w in enumerate(wfsetsub.waveforms) if w], key= lambda x: x[0])
timestamps[0]

In [ ]:
wfsort = dict( sorted( wfsetch[endpoint].items(), key=lambda x: np.argwhere(x[0] == np.array(validchannels[endpoint]))[0] ) )
for ch, w in wfsort.items():
    counts = {}
    timestamps = sorted([ wtf.timestamp for wtf in w.waveforms])
    for i, wft in enumerate(wfsetch[endpoint][ch].waveforms):
        counts[wft.daq_window_timestamp] = counts.get(wft.daq_window_timestamp, 0) + 1
        # if wft.timestamp == 110611965912507851:
        #     print(i, ch, 'aaaahh')
    triggerrate_per_daq_window = np.mean( [ v for v in counts.values() ] )
    print(ch, dict_uniqch_to_module[strUch(endpoint, ch)], triggerrate_per_daq_window)


In [ ]:
a=1
ch = 34
# print(wfsetch[endpoint][ch].waveforms[a].timestamp)
ttest = [ wft.timestamp - wft.daq_window_timestamp for wft in wfsetch[endpoint][ch].waveforms if wft.record_number == a ]
daqstamp = [ wft.daq_window_timestamp for wft in wfsetch[endpoint][ch].waveforms if wft.record_number == a ]
wfv = [ wft.adcs for wft in wfsetch[endpoint][ch].waveforms if wft.record_number == a ]
times = [ np.linspace(t, t+wfsetsub.points_per_wf-1, wfsetsub.points_per_wf) for t in ttest ]
print(len(times))
for w, t, dt in zip(wfv, times, daqstamp):
    plt.plot(t, w, '.')
    print(t[0], dt)
# plt.ylim(2500,2800)
# plt.xlim(70000,80000)

In [ ]:
a=0

wft = wfsetsub.waveforms[a]
apply_derivativeWf(wft)
apply_moving_averageWf(wft, width=12)
apply_moving_averageWf(wft, width=12)
apply_derivativeWf(wft, again=True)
# wft.derivative_2 = moving_average(wft.derivative_2, width=10)
peaks = find_peaks_from_derivative_2(wft.derivative, wft.derivative_2, timestamp=wft.timestamp, daq_window_timestamp=wft.daq_window_timestamp)
timemin = 0
timemax = 1024
    
startt = wft.timestamp - wft.daq_window_timestamp
time = np.linspace( startt, startt+len(wft.adcs)-1, len(wft.adcs-1))
argmin = np.argwhere(time==timemin)[0][0] if timemin else None
argmax = np.argwhere(time==timemax)[0][0] if timemax else None

timecut = slice(argmin, argmax)
print(timecut)
time = time[timecut]
plt.plot(time, wft.adcs[timecut] - wft.analyses['std'].result['baseline'])
plt.plot(time, wft.derivative[timecut])
plt.title(f'{dict_uniqch_to_module[strUch(endpoint, wft.channel)]}, Run: {run}, Record: {wft.record_number}')
# plt.plot(time, wft.derivative_2[timecut])
# plt.axvline(-wft.time_offset, c='r', ls='--')
# plt.ylim(-350,3000)
# plt.xlim(-500,3000)
dict_uniqch_to_module[strUch(endpoint, wft.channel)]
# plt.xlim(35500,37500)
for ppos, pamp in peaks:
    # if ppos > 2000:
    #     break 
    if timemin and ppos < timemin:
        continue
    if timemax and ppos > timemax:
        continue
    plt.plot(ppos, pamp, 'o', c='k')
# plt.savefig('example.png')


In [ ]:
matplotlib_plot_WaveformSetGrid(wfset=wfsetsub, plot_function=plot_counts, detector=group1+group2+group3+group4, cols=2, figsize=(14,32), func_params={"run": 42410, "rates_results": rates_results})
# plt.savefig(f'{dettype}_rate_run{run:06d}.png')


In [ ]:


ratioparams = dict(
    rates_results = rates_results,
    run_more = 42417,
    run_less = 42410
)

# matplotlib_plot_WaveformSetGrid(wfset=wfset_peaks, plot_function=plot_ratio, detector=group2+group3+group4, cols=2, figsize=(14,32), func_params=ratioparams)
# plt.savefig(f'ratio_run{ratioparams["run_more"]:06d}_over_run{ratioparams["run_less"]:06d}.png')

ratioparams = dict(
    rates_results = rates_results,
    runs_to_plot = []
)

matplotlib_plot_WaveformSetGrid(wfset=wfsetsub, plot_function=plot_runs, detector=group1+group2+group3+group4, cols=2, figsize=(14,32), func_params=ratioparams)
if not ratioparams['runs_to_plot']:
    runsname = list(list(rates_results.items())[0][1].keys())
else:
    runsname = ratioparams['runs_to_plot']
runsname = [ f'run{x:06d}' for x in runsname ]
runsname = '_'.join(runsname)
filenameruns=f'rates_{dettype}_{runsname}.png'
print(filenameruns)
plt.savefig(filenameruns)
    